In [ ]:
# MIT License (https://opensource.org/licenses/MIT)
# Copyright 2018 Ryan Hausen

![morpheus](https://cdn.jsdelivr.net/gh/morpheus-project/morpheus/morpheus.svg)



This notebook can be run interactively in Google Colab
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/morpheus-project/morpheus/blob/master/examples/example_array.ipynb)

*note: if you're running in Colab, you can enable GPU/TPU acceleration, by going to Edit->Notebook settings->Hardware accelerator*

# Table of contents
1. [Introduction](#introduction)
2. [Cataloging](#cataloging)
3. [Classifiying An Image](#classifying_an_image)
4. [Making A Segmentation Map](#making_a_segmentation_map)
4. [Colorizing A Morphological Classification](#colorizing_a_morphological_classification)

## Introduction <a name="introduction"></a>

This is a walk through for basic image classification using 
[Morpheus](https://github.com/morpheus-project/morpheus). Moprheus leverages a neural network 
to simultaneously perform source identification and 
morphological classification at the pixel level. The primary way to interact with Morpheus
is via the [Classifier](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier) 
class.

In [ ]:
# if your running on Colab you need to install the package.
!pip install morpheus-astro

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

from morpheus.classifier import Classifier
from morpheus.data import example

%matplotlib inline

### First get a sample image to examine.

This is a sample taken from GOODS which includes multiple sources with different morphologies.

In [ ]:
h, j, v, z = example.get_sample()

In [ ]:
f, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
axes = np.array(axes).flatten()

for ax, arr, band in zip(axes, [h,j,v,z], 'HJVZ'):
    ax.set_title(f'{band} Image')
    ax.imshow(arr, origin='lower', cmap='gray')
plt.show()

## Classify Images <a name="classification"></a>

To classify an image use [Classifer.classify](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.classify).
The arguments can be either a file path or a numpy array. The output is a dictionary that contains the
classifications as numpy arrays

In [ ]:
morphs = Classifier.classify_arrays(h=h, j=j, v=v, z=z)

In [ ]:
f, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 15))
axes = np.array(axes).flatten()

for i, k in enumerate(morphs):
    axes[i].set_title(f'Output: {k}')
    axes[i].imshow(morphs[k], origin='lower', vmin=0, vmax=1, cmap='magma')
plt.show()

## Segmentation Map <a name="segmap"></a>

After an image has been classified, a segmap can be created using [Classifier.segmap_from_classified](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.segmap_from_classifed).
For more information about the algorithm see the documentation for the function.

In [ ]:
# the mask tells the segmentation mapping algorithm to ignore
# the areas unclassified as a result of the windowing classification
# method used.
mask = np.zeros_like(h, np.int)
mask[5:-5, 5:-5] = 1

segmap = Classifier.segmap_from_classified(morphs, h, mask=mask)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(segmap, origin='lower', cmap='gray')
plt.grid(None)
plt.show()

## Cataloging <a name="cataloging"></a>

Morpheus provides a cataloging functionality via [Classifier.catalog_from_classified](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.catalog_from_classified)
for images that will return all of the detected sources and their morphological classifications.


In [ ]:
catalog = Classifier.catalog_from_classified(h=h, j=j, v=v, z=z)

In [ ]:
print('Source ID\tLocation(y,x)\tMorphology:[Sph,Dsk,Irr,Ps]')
for source in catalog['catalog']:
    _id = source['id']
    loc = source['location']
    morph = np.round(source['morphology'], decimals=2) # round for readability
    print('{}\t\t{}\t{}'.format(_id, loc, morph))


## Classifying image <a name="classifying_an_image"></a>

Use [Classifer.classify_arrays](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.classify_arrays) to classify a nummpy array in memory. 
See documentation for classifying images from disk.

The output is a dictionary that contains a mapping for each pixel which representss the 
probability that a pixel belongs to one of the following classes:

- Spheroid
- Disk
- Irregular
- Point Source (Compact)
- Background

It also contains a mapping for `n`, which indicates how many times a classification was
recorded for that pixel

In [ ]:
morphs = Classifier.classify_arrays(h=h, j=j, v=v, z=z)

In [ ]:
f, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 15))
axes = np.array(axes).flatten()

for i, k in enumerate(morphs):
    axes[i].set_title(f'Output: {k}')
    axes[i].imshow(morphs[k], origin='lower', vmin=0, vmax=1, cmap='magma')
plt.show()

## Making a Segmentation Map <a name="making_a_segmentation_map"></a>

Use [Classifier.make_segmap](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.make_segmap) 
to generate a segmentation map using the output and the H band flux. Its important to note that because
of the windowing technique used by Morpheus, the outermost 5 pixels of the image are not classified and are
indicated as such with the value -1 in the segmentation map

In [ ]:
# the mask tells the segmentation mapping algorithm to ignore
# the areas unclassified as a result of the windowing classification
# method used.
mask = np.zeros_like(h, np.int)
mask[5:-5, 5:-5] = 1

segmap = Classifier.make_segmap(morphs, h, mask=mask)

plt.figure(figsize=(10, 10))
plt.imshow(segmap, origin='lower', cmap='gray')
plt.grid(None)
plt.show()

## Colorizing A Morphological Classification <a name="colorizing_a_morphological_classification"></a>

Use [Classifier.colorize_rank_vote_output](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.colorize_rank_vote_output)  to make an RGB. The colors in the output have the following meanings:

Red = Spheroid
Blue = Disk
Green = Irregular
Yellow = Point Source (compact)
Back = Background

Note when Morpheus isn't doesn't output a strong classification for any one class they are colored white.

For more inforamtion on the coloring scheme see the [documentation](https://morpheus-astro.readthedocs.io/en/latest/source/morpheus.html#morpheus.classifier.Classifier.colorize_rank_vote_output).

In [ ]:
rgb = Classifier.colorize_rank_vote_output(morphs)
plt.figure(figsize=(10, 10))
plt.imshow(rgb, origin='lower')
plt.grid(None)
plt.show()